In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime


import yfinance as yf
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

import sambo
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy

from backtesting.test import SMA, GOOG

# backtesting.set_bokeh_output(notebook=True)
            
import itertools 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True

C:\Users\ASUS\AppData\Local\Temp\ipykernel_13088\2595710202.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
d:\Binance Tradebot\.venv\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
from utils.loader import *
from utils.signals import *
from utils.trade import *
from utils.strategy import *

In [36]:
# now = datetime.today().strftime('%Y-%m-%d')

loader = DataLoader(ticker='AAPL', start='2010-01-01', end='2025-12-31', freq='1d', test_size=0.2)
loader.run()
# test     

[*********************100%***********************]  1 of 1 completed


In [41]:
def OBV(price: pd.Series, volume: pd.Series) -> pd.Series:
    # Calculate the price difference
    # diff = np.diff(price)

    diff = np.insert(np.diff(price), 0, 0)

    # Compute OBV changes: add volume if price increases, subtract if it decreases,
    # and 0 if there's no change
    obv_changes = np.where(diff > 0, volume, np.where(diff < 0, -volume, 0))
    
    # Create a pandas Series with the same index as the price and calculate cumulative sum
    obv = pd.Series(obv_changes).cumsum()
    
    return obv

class EMACross_test(Strategy):
    '''
    
    '''

    short_duration = 5  # Default values, can be overridden
    long_duration = 10
    atr_period = 14
    id = 0
    take_profit_ratio = 1.3
    take_profit_next_order = 1.10
    stop_loss_ratio = 0.8

    stop_loss_duration = 3

    lookback = 3
    atr_multiplier = 1

    def init(self):
        price = self.data.Close
        volume = self.data.Volume
        data = self.data
        self.ma1 = self.I(EMA, price, self.short_duration, plot=True, overlay=True)
        self.ma2 = self.I(EMA, price, self.long_duration, plot=True, overlay=True)
        self.bbw = self.I(BBW, price, plot=False, overlay=False)
        # self.atr = self.I(ATR, price, plot=True, overlay=False)
        self.obv = self.I(OBV, self.data.Close, self.data.Volume)
        self.obv_ma = self.I(EMA, self.obv, 3, plot = True, overlay = False)

        self.volume_ma = self.I(EMA, volume, 5, plot = True, overlay = False)

        self.atr = self.I(ATR, self.data, self.atr_period)
        self.previous_low = self.I(previous_low, self.data.Low, self.stop_loss_duration, plot = False)
        self.previous_high = self.I(previous_high, self.data.High, self.stop_loss_duration, plot = False)

    def next(self):
        entry_price = self.data.Close[-1]

        long_stop_loss = min(self.previous_low[-1], entry_price - self.atr_multiplier*self.atr[-1])
        long_stop_loss_next = min(self.previous_low[-1], entry_price - self.atr_multiplier*(1/2)*self.atr[-1])

        for trade in self.trades:
            if trade.tag == "Long":
                trade.sl = long_stop_loss
            if trade.tag == "Next":
                trade.sl = long_stop_loss_next


        past_max = max(self.bbw[-self.lookback-1:-1])  # Maximum in last 4 periods
        current_bbw = self.bbw[-1]

        count_green = sum(self.data.Color[-2:])
        
        #add if current OBV > average OBV -> buy
        if crossover(self.ma1, self.ma2) and count_green >= 1:
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Long {self.id}')
            self.id += 1
        
        #for this kind of order, set take_profit smaller (mua duoi), sl tighter
        elif self.ma1[-1] > self.ma2[-1] and self.ma1[-2] > self.ma2[-2] and count_green >= 2 and not self.position:
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_next_order, sl=entry_price*self.stop_loss_ratio, tag=f'Next')
            self.id += 1
            
        elif crossover(self.ma2, self.ma1):
            for trade in self.trades:
                trade.close()

        # elif crossover(self.ma2, self.ma1):
        #     for trade in self.trades:
        #         if trade.tag ==f'Long {self.id-1}':
        #             trade.close()

strategy = EMACross_test
# strategy = BollingerBound
bt = BackTrader(data=loader.data)

# params = {
#     'short_duration': 5,
#     'long_duration': 10
# }
params = {'take_profit_ratio': 1.3}
bt.evaluate(data=bt.test_data, strategy=strategy, params=params, order_size=0.9999, plot=True)
# bt.trades.head()

Start                     2022-03-03 00:00:00
End                       2025-03-21 00:00:00
Duration                   1114 days 00:00:00
Exposure Time [%]                     57.5718
Equity Final [$]                  14482.09358
Equity Peak [$]                   16639.97601
Commissions [$]                    1278.56976
Return [%]                           44.82094
Buy & Hold Return [%]                28.77407
Return (Ann.) [%]                    12.95633
Volatility (Ann.) [%]                19.93998
CAGR [%]                              8.73815
Sharpe Ratio                          0.64977
Sortino Ratio                          1.1213
Calmar Ratio                          0.79306
Max. Drawdown [%]                   -16.33704
Avg. Drawdown [%]                    -4.11484
Max. Drawdown Duration      182 days 00:00:00
Avg. Drawdown Duration       38 days 00:00:00
# Trades                                   26
Win Rate [%]                         42.30769
Best Trade [%]                    

In [42]:
# grid_search = {
#     'short_duration': range(2, 4),
#     'long_duration': range(5, 11)
# }
grid_search={'take_profit_ratio': [1.3], 'stop_loss_ratio': [0.9]}

bt.cross_val(strategy=strategy, train_size=240, test_size=240, step_size=240, order_size=0.999, commission=0.002, grid=grid_search)

{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
7627.160763078552
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
15390.73427357381
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
10636.57367822256
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
10074.93163351444
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
10551.039019804055
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
11598.688510977065
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
13891.662807403856
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
11978.817381525372
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
12259.805196492118
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
15636.392511561502
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
12216.116481280393


11987.44747794852

In [6]:
for point in train:
    if point < 0:
        print(point)
        
    print(point)

NameError: name 'train' is not defined